In [6]:
import json
import sagemaker
import boto3

iam = boto3.client('iam')
role = iam.get_role(RoleName='sagemaker-dlc-demo')['Role']['Arn']



In [11]:
llm_image = "754289655784.dkr.ecr.us-east-1.amazonaws.com/test-tgi:vllm-0.12.0-gpu-py312-cu129-ubuntu22.04-sagemaker"

In [19]:

import json
from sagemaker.huggingface import HuggingFaceModel

# sagemaker config
instance_type = "ml.g5.12xlarge"
health_check_timeout = 900


# Define Model and Endpoint configuration parameter
config = {
  'SM_VLLM_MODEL': "Qwen/Qwen3-4B-Thinking-2507", # model_id from hf.co/models
  'SM_VLLM_MAX_MODEL_LEN': "2048",  # Max length of input text
  'SM_VLLM_HOST': "0.0.0.0",  # Required for SageMaker networking
  #'HUGGING_FACE_HUB_TOKEN': "<REPLACE WITH YOUR TOKEN>"
}


# create HuggingFaceModel with the image uri
llm_model = HuggingFaceModel(
  role=role,
  image_uri=llm_image,
  env=config,
  name="qwen3-4b-thinking-2507-demo-endpoint"
)

In [20]:
# Deploy model to an endpoint
# https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy
llm = llm_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=health_check_timeout,
  inference_ami_version="al2-ami-sagemaker-inference-gpu-3-1", 
  wait=True
)


Using already existing model: qwen3-4b-thinking-2507-demo-endpoint


--------------!

In [5]:
# Prompt to generate
messages=[
    { "role": "user", "content": "Give me a short introduction to large language model" }
  ]

# Generation arguments
parameters = {
    "top_p": 0.6,
    "temperature": 0.9,
    "max_tokens": 128,
}


In [6]:
chat = llm.predict({"messages" :messages, **parameters})

print(chat["choices"][0]["message"]["content"].strip())

Okay, the user asked for a short introduction to large language models. Let me start by recalling what I know. LLMs are a big topic in AI, so I need to keep it concise but informative. 

First, I should define what an LLM is. They're AI models trained on massive amounts of text data. The key points are their size—billions or trillions of parameters—and their ability to generate human-like text. 

Wait, the user might not know terms like "parameters." I should explain it simply. Maybe say they're huge neural networks that learn patterns from text. Also, mention common tasks:


In [14]:
llm.delete_model()
llm.delete_endpoint()

NameError: name 'llm' is not defined

In [15]:
llm_model.delete_model()